In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row
from pyspark.sql.functions import max as spark_max
from delta.tables import DeltaTable
import os

# -------------------
# Spark session
# -------------------
spark = (
    SparkSession.builder
        .appName("Delta S3 Hive Auto Insert")
        .master("local[*]")
        .config("spark.jars.packages",
                "io.delta:delta-spark_2.13:4.0.0,org.apache.hadoop:hadoop-aws:3.4.1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        # MinIO / S3
        .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
        .config("spark.hadoop.fs.s3a.access.key", "admin")
        .config("spark.hadoop.fs.s3a.secret.key", "password")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        .config("spark.sql.warehouse.dir", "s3a://warehouse/")
        # Hive Metastore
        .config("spark.hadoop.hive.metastore.uris", "thrift://localhost:9083")
        .config("spark.sql.catalogImplementation", "hive")
        .getOrCreate()
)

print("Spark session started")

# -------------------
# Define schema
# -------------------
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
])

# -------------------
# Create Hive database if missing
# -------------------
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/28 20:27:57 WARN Utils: Your hostname, ubuntu2020-virtual-machine, resolves to a loopback address: 127.0.1.1; using 192.168.87.134 instead (on interface ens33)
25/12/28 20:27:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ubuntu2020/.ivy2.5.2/cache
The jars for the packages stored in: /home/ubuntu2020/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ac8db8e9-14c5-4eb7-98fc-e64b19633d6e;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
	f

Spark session started


25/12/28 20:28:09 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


DataFrame[]